In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'advanced-rag'
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

In [ ]:
from langchain.text_splitter import TokenTextSplitter, SentenceTransformersTokenTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

with open("HoChiMinh.txt", "r", encoding="utf-8") as f:
    docs = f.read()
##1 - 0 - 1000 , 800 - 1800

# Split - Chunking
# Khởi tạo bộ chia văn bản
# Khởi tạo bộ chia văn bản
text_splitter = SentenceTransformersTokenTextSplitter(
    chunk_size=2000,  # Số token mỗi đoạn
    chunk_overlap=50,  # Overlap
    model_name="BAAI/bge-m3"  # Model tokenizer sử dụng
)

# Chia nhỏ văn bản
texts = text_splitter.create_documents([docs])

# Kiểm tra kết quả
print(f"Số đoạn sau khi chia: {len(texts)}")
print(texts[0])

Số đoạn sau khi chia: 4
page_content='Mục từ "Nguyễn Ái Quốc" dẫn đến bài này. Xin đọc về bút hiệu Nguyễn Ái Quốc tại Nguyễn Ái Quốc (bút hiệu). Đối với các định nghĩa khác, xem Hồ Chí Minh (định hướng). Hồ Chí Minh Hồ Chí Minh vào năm 1946 Chủ tịch Đảng Lao động Việt Nam Nhiệm kỳ 19 tháng 2 năm 1951 – 2 tháng 9 năm 1969 Tổng Bí thư Trường Chinh Lê Duẩn Tiền nhiệm Mới lập Kế nhiệm Bãi bỏ Tổng Bí thư Ban Chấp hành Trung ương Đảng Lao động Việt Nam Nhiệm kỳ 1 tháng 11 năm 1956 – 10 tháng 9 năm 1960 Tiền nhiệm Trường Chinh Kế nhiệm Lê Duẩn Chủ tịch nước Việt Nam Dân chủ Cộng hòa Nhiệm kỳ 2 tháng 9 năm 1945 – 2 tháng 9 năm 1969 Tiền nhiệm Mới lập Kế nhiệm Tôn Đức Thắng Thủ tướng Việt Nam Dân chủ Cộng hòa Nhiệm kỳ 2 tháng 9 năm 1945 – 20 tháng 9 năm 1955 Tiền nhiệm Mới lập Kế nhiệm Phạm Văn Đồng Bộ trưởng Bộ Ngoại giao Nhiệm kỳ 28 tháng 8 năm 1945 – 2 tháng 3 năm 1946 Tiền nhiệm Mới lập Kế nhiệm Nguyễn Tường Tam Nhiệm kỳ 3 tháng 11 năm 1946 – tháng 3 năm 1947 Tiền nhiệm Nguyễn Tường Tam Kế 

In [ ]:
# Embed
model_name = "BAAI/bge-m3"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

vectorstore = FAISS.from_documents(documents=texts, 
                                    embedding=hf_embeddings)

retriever = vectorstore.as_retriever() # Dense Retrieval - Embeddings/Context based

C:\Users\Hokta\AppData\Local\Temp\ipykernel_6788\251197425.py:5: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceBgeEmbeddings(


In [ ]:
retriever.invoke("Chủ tịch Hồ Chí Minh làm chủ tịch nước nhiệm kỳ nào ?")

[Document(id='b1f922a9-8ba8-4ecc-873b-f99213013b3c', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Retrieval model: surfaces the context to inform the agent’s behavior, according to relevance, recency and importance.'),
 Document(id='f49fd922-52bb-4970-b602-1b98c3d73b07', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 13. The generative agent architecture. (Image source: Park et al. 2023)'),
 Document(id='4742cbe0-f930-4f1f-b13f-69e2e18e9f95', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='in reinforcement learning tasks, where an algorithm is encapsulated in a long history-conditioned policy. Considering that an agent interacts with the environment many times and in each episode the agent gets a little better, AD concatenates this learning history and feeds that into the model.'),
 Document(id='79fefb05-3740-4a16-b844-f52e418c4ab6', metadata={'sourc

In [ ]:
vectorstore.save_local("faiss_index")

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

# Khởi tạo mô hình embedding giống như khi lưu
model_name = "BAAI/bge-m3"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

# Load FAISS index từ thư mục đã lưu
vectorstore = FAISS.load_local("faiss_index", hf_embeddings, allow_dangerous_deserialization=True)

# Kiểm tra thử bằng cách truy vấn
query = "Hồ Chí Minh là ai?"
retriever = vectorstore.as_retriever()
results = retriever.get_relevant_documents(query)

# In kết quả
for i, doc in enumerate(results):
    print(f"Document {i+1}: {doc.page_content}")
